# CS235F25 HW 2

**Dataset:** Airbnb

# Imports

In [98]:
# [INSERT CODE TO IMPORT NECESSARY PYTHON PACKAGES HERE]

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder

# 0. Load Data / Data Exploration

In [99]:
trainDF = pd.read_csv('train.csv')
testDF = pd.read_csv('test.csv')

/var/folders/7m/jkfxndv53l300tq3qk8xlcs00000gn/T/ipykernel_44229/1745673260.py:1: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  trainDF = pd.read_csv('train.csv')


In [100]:
# Inspected the data type of each feature
trainDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26830 entries, 0 to 26829
Data columns (total 65 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                26830 non-null  int64  
 1   name                              26823 non-null  object 
 2   summary                           25821 non-null  object 
 3   space                             18483 non-null  object 
 4   description                       26580 non-null  object 
 5   experiences_offered               26830 non-null  object 
 6   neighborhood_overview             16010 non-null  object 
 7   notes                             10740 non-null  object 
 8   transit                           16719 non-null  object 
 9   access                            15465 non-null  object 
 10  interaction                       14972 non-null  object 
 11  house_rules                       16051 non-null  object 
 12  host

In [101]:
# Checked for NULL, NA, and Duplicated row for the target variable 

print(f"Number of NA cells: {trainDF['price'].isna().sum()}")
print(f"Number of NULL cells: {trainDF['price'].isnull().sum()}")
print(f"Number of duplicated cells: {trainDF['price'].duplicated().sum()}") # keep duplicated rows because some Airbnb listings could have the same price

start_sample_count = trainDF.shape[0]
print(f"Number of samples: {start_sample_count}")

Number of NA cells: 0
Number of NULL cells: 0
Number of duplicated cells: 26301
Number of samples: 26830


In [102]:
# Inspect features for duplicates

# Turned off viewing limit
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

print(f"Number of duplicated rows: {trainDF.duplicated().sum()}")


Number of duplicated rows: 0


In [103]:
# Inspect features for NULL/ NA values
print(trainDF.isnull().sum()) # count of na values

id                                      0
name                                    7
summary                              1009
space                                8347
description                           250
experiences_offered                     0
neighborhood_overview               10820
notes                               16090
transit                             10111
access                              11365
interaction                         11858
house_rules                         10779
host_id                                 0
host_name                               4
host_since                              4
host_location                          87
host_about                          10497
host_response_time                  12998
host_response_rate                  12998
host_acceptance_rate                26830
host_is_superhost                       4
host_neighbourhood                   3774
host_listings_count                     4
host_verifications                

In [104]:
# Express NA values as percentages
((trainDF.isna().sum()/ len(trainDF)) * 100).round(2)

id                                    0.00
name                                  0.03
summary                               3.76
space                                31.11
description                           0.93
experiences_offered                   0.00
neighborhood_overview                40.33
notes                                59.97
transit                              37.69
access                               42.36
interaction                          44.20
house_rules                          40.18
host_id                               0.00
host_name                             0.01
host_since                            0.01
host_location                         0.32
host_about                           39.12
host_response_time                   48.45
host_response_rate                   48.45
host_acceptance_rate                100.00
host_is_superhost                     0.01
host_neighbourhood                   14.07
host_listings_count                   0.01
host_verifi

## Removed N/A values

In [105]:
# Identified that "square_feet" and "host_acceptance_rate" are almost completely NA (98.96% and 100% NA)
# Dropped all features that are over 50% NA ("Notes")
# Dropped "experience_offered" as all cells are listed with "None" instead of NA value

trainDF = trainDF.drop(columns = ["experiences_offered","square_feet", "host_acceptance_rate", "notes"], axis=1)
trainDF.head()

,id,name,summary,space,description,neighborhood_overview,transit,access,interaction,house_rules,host_id,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,country_code,country,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,15156865,"East 12th st, Lux Studio in Greenwich Village**",A 14-story luxury doorman building at the cent...,NaN,A 14-story luxury doorman building at the cent...,NaN,NaN,NaN,NaN,- Building management requires each new guest ...,22541573,Ken,2014-10-14,"New York, New York, United States","I work for Furnished Quarters, the largest pro...",within an hour,99%,t,Upper East Side,155.0,"['email', 'phone', 'reviews', 'jumio', 'kba', ...",t,t,East Village,Manhattan,New York,NY,10003.0,New York,US,United States,Serviced apartment,Entire home/apt,2,1.0,0.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,G...",199.0,1,$0.00,30,1125,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,f,flexible,f,f,98,NaN
1,14744912,"sunny airy bohemian rm, private bath! hip 'shwick",A full private bathroom makes you very comfort...,"Large square room, full private bathroom in ha...",A full private bathroom makes you very comfort...,"Interesting street art, people watching, aweso...",We're less than one block away from the L trai...,coin-op laundry room in basement.,I will be available in person or online if you...,"For any questions you must contact us, do not ...",948164,Theodora,2011-08-09,"New York, New York, United States","I am a young professional in Brooklyn, NY. I h...",NaN,NaN,f,NaN,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,Bushwick,Brooklyn,Brooklyn,NY,11237.0,New York,US,United States,Apartment,Private room,3,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",80.0,1,$10.00,2,300,4,2016-09-06,2018-04-30,95.0,10.0,10.0,10.0,10.0,10.0,10.0,t,f,strict_14_with_grace_period,f,f,1,0.15
2,20280692,"2 bedroom New York, Queens",Quiet 2 bedroom apartment minutes from Astoria...,NaN,Quiet 2 bedroom apartment minutes from Astoria...,NaN,NaN,NaN,NaN,NaN,46407533,Alba,2015-10-12,"New York, New York, United States",NaN,within an hour,100%,f,NaN,1.0,"['email', 'phone', 'reviews']",t,f,Ditmars Steinway,Queens,Queens,NY,11105.0,New York,US,United States,Apartment,Entire home/apt,4,1.0,2.0,3.0,Real Bed,"{TV,""Cable TV"",Wifi,""Air conditioning"",Kitchen...",160.0,1,$0.00,5,1125,2,2017-09-03,2018-09-02,100.0,10.0,10.0,8.0,9.0,9.0,10.0,t,f,flexible,f,f,1,0.14
3,3716193,Beautiful 2 Bedroom Townhouse,"Sunny, spacious, beautifully renovated 2 bedro...","Sunny, spacious, beautifully renovated 2 bedro...","Sunny, spacious, beautifully renovated 2 bedro...","Beautiful brownstones, strong community, block...",The A/C train on Nostrand Avenue is 7 minutes ...,"For your cooking needs, the kitchen is fully e...",Depends on the guest's preference but more tha...,"Please don't order pay-per-view movies, we hav...",18970667,"Erin, Avi, Kaleb & Shiloh",2014-07-27,"New York, New York, United States",We are a fun loving couple living in Bed-stuy ...,within an hour,100%,f,Bedford-Stuyvesant,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,Bedford-Stuyvesant,Brooklyn,Brooklyn,NY,11216,New York,US,United States,Apartment,Entire home/apt,6,1.0,2.0,3.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning..."

In [106]:
# Dropped columns that include textual summaries that vary depending on row (removed unstructured data with varying context)
# These columns include: "name", "summary", "space", "description", "neighborhood", "transit", "access", "interaction", "house_rules", "host_name", "host_about"
trainDF = trainDF.drop(columns=["name", "summary", "space", "description", "neighborhood_overview", "transit", "access", "interaction", "house_rules", "host_name", "host_about"], axis=1)
trainDF.head()

,id,host_id,host_since,host_location,host_response_time,host_response_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,country_code,country,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,15156865,22541573,2014-10-14,"New York, New York, United States",within an hour,99%,t,Upper East Side,155.0,"['email', 'phone', 'reviews', 'jumio', 'kba', ...",t,t,East Village,Manhattan,New York,NY,10003.0,New York,US,United States,Serviced apartment,Entire home/apt,2,1.0,0.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,G...",199.0,1,$0.00,30,1125,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,f,flexible,f,f,98,NaN
1,14744912,948164,2011-08-09,"New York, New York, United States",NaN,NaN,f,NaN,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,Bushwick,Brooklyn,Brooklyn,NY,11237.0,New York,US,United States,Apartment,Private room,3,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",80.0,1,$10.00,2,300,4,2016-09-06,2018-04-30,95.0,10.0,10.0,10.0,10.0,10.0,10.0,t,f,strict_14_with_grace_period,f,f,1,0.15
2,20280692,46407533,2015-10-12,"New York, New York, United States",within an hour,100%,f,NaN,1.0,"['email', 'phone', 'reviews']",t,f,Ditmars Steinway,Queens,Queens,NY,11105.0,New York,US,United States,Apartment,Entire home/apt,4,1.0,2.0,3.0,Real Bed,"{TV,""Cable TV"",Wifi,""Air conditioning"",Kitchen...",160.0,1,$0.00,5,1125,2,2017-09-03,2018-09-02,100.0,10.0,10.0,8.0,9.0,9.0,10.0,t,f,flexible,f,f,1,0.14
3,3716193,18970667,2014-07-27,"New York, New York, United States",within an hour,100%,f,Bedford-Stuyvesant,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,Bedford-Stuyvesant,Brooklyn,Brooklyn,NY,11216,New York,US,United States,Apartment,Entire home/apt,6,1.0,2.0,3.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",100.0,2,$15.00,3,400,151,2014-08-09,2018-10-21,93.0,10.0,9.0,10.0,10.0,9.0,9.0,t,f,strict_14_with_grace_period,f,f,1,2.92
4,22410246,54586794,2016-01-14,"Paris, Île-de-France, France",NaN,NaN,f,Upper East Side,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,Upper East Side,Manhattan,New York,NY,10022,New York,US,United States,Apartment,Entire home/apt,4,2.0,2.0,2.0,Real Bed,"{TV,Wifi,""Air conditioning"",Kitchen,Gym,Elevat...",250.0,1,$0.00,2,7,1,2017-12-31,2017-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,f,flexible,f,f,1,0.10


In [107]:
# Dropped features that have little to no impact of prediction: "id", "host_id", "is_business_travel_ready"(all f)
trainDF = trainDF.drop(columns=["id", "host_id", "is_business_travel_ready"], axis=1)
trainDF.head()

,host_since,host_location,host_response_time,host_response_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,country_code,country,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,2014-10-14,"New York, New York, United States",within an hour,99%,t,Upper East Side,155.0,"['email', 'phone', 'reviews', 'jumio', 'kba', ...",t,t,East Village,Manhattan,New York,NY,10003.0,New York,US,United States,Serviced apartment,Entire home/apt,2,1.0,0.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,G...",199.0,1,$0.00,30,1125,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,flexible,f,f,98,NaN
1,2011-08-09,"New York, New York, United States",NaN,NaN,f,NaN,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,Bushwick,Brooklyn,Brooklyn,NY,11237.0,New York,US,United States,Apartment,Private room,3,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",80.0,1,$10.00,2,300,4,2016-09-06,2018-04-30,95.0,10.0,10.0,10.0,10.0,10.0,10.0,t,strict_14_with_grace_period,f,f,1,0.15
2,2015-10-12,"New York, New York, United States",within an hour,100%,f,NaN,1.0,"['email', 'phone', 'reviews']",t,f,Ditmars Steinway,Queens,Queens,NY,11105.0,New York,US,United States,Apartment,Entire home/apt,4,1.0,2.0,3.0,Real Bed,"{TV,""Cable TV"",Wifi,""Air conditioning"",Kitchen...",160.0,1,$0.00,5,1125,2,2017-09-03,2018-09-02,100.0,10.0,10.0,8.0,9.0,9.0,10.0,t,flexible,f,f,1,0.14
3,2014-07-27,"New York, New York, United States",within an hour,100%,f,Bedford-Stuyvesant,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,Bedford-Stuyvesant,Brooklyn,Brooklyn,NY,11216,New York,US,United States,Apartment,Entire home/apt,6,1.0,2.0,3.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",100.0,2,$15.00,3,400,151,2014-08-09,2018-10-21,93.0,10.0,9.0,10.0,10.0,9.0,9.0,t,strict_14_with_grace_period,f,f,1,2.92
4,2016-01-14,"Paris, Île-de-France, France",NaN,NaN,f,Upper East Side,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,Upper East Side,Manhattan,New York,NY,10022,New York,US,United States,Apartment,Entire home/apt,4,2.0,2.0,2.0,Real Bed,"{TV,Wifi,""Air conditioning"",Kitchen,Gym,Elevat...",250.0,1,$0.00,2,7,1,2017-12-31,2017-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,flexible,f,f,1,0.10


In [108]:
# [host_response_rate] 

# Convert features from precentages (strings) to floats
trainDF["host_response_rate"] = trainDF["host_response_rate"].str.rstrip("%").astype(float)

median_response_rate = trainDF.loc[trainDF["host_response_rate"] < 100, "host_response_rate"].median() # since the distribution heavily-right skewed, median is calculated with values less than 100
trainDF["missing_host_response_rate"] = trainDF["host_response_rate"].isna().astype(int) # created another column to account for missing host_response_rates values (1 for present, 0 for missing)

# Replaced "host_response_rate" NA values with median value
trainDF["host_response_rate"] = trainDF["host_response_rate"].fillna(median_response_rate)


# "missing_host_response_rate" highlights rows that were previously NA/ NULL
trainDF[["host_response_rate", "missing_host_response_rate"]].head(5)

,host_response_rate,missing_host_response_rate
0,99.0,0
1,86.0,1
2,100.0,0
3,100.0,0
4,86.0,1


In [109]:
# [host_response_time]

# Created "missing_host_response_time" to account for missing host_response_rates values (1 for present, 0 for missing)
trainDF["missing_host_response_time"] = trainDF["host_response_time"].isna().astype(int)
display(trainDF[["missing_host_response_time"]].value_counts())

# Add a "missing" cell for NA values in "host_response_time"
trainDF["host_response_time"] = trainDF["host_response_time"].fillna("missing")

# Encoded values using Ordinal encoder
categories = [["missing", "within an hour", "within a few hours", "within a day", "a few days or more"]]
oc = OrdinalEncoder(categories=categories)
trainDF["host_response_time"] = oc.fit_transform(trainDF[["host_response_time"]])

trainDF["host_response_time"].value_counts()


missing_host_response_time
0                             13832
1                             12998
Name: count, dtype: int64

host_response_time
0.0    12998
1.0     8224
2.0     2967
3.0     2210
4.0      431
Name: count, dtype: int64

In [110]:
# Drop NA values from the rest of the dataset
trainDF= trainDF.dropna()
end_sample_count = trainDF.shape[0]
print(f"Percent of dataset retained after data cleaning: {(round(end_sample_count/start_sample_count, 2)*100)}%")

Percent of dataset retained after data cleaning: 66.0%


## Data Preprocessing

In [125]:
trainDF.head()

,host_since,host_location,host_response_time,host_response_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,country_code,country,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,missing_host_response_rate,missing_host_response_time
3,2014-07-27,"New York, New York, United States",1.0,100.0,0.0,Bedford-Stuyvesant,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",1.0,1.0,Bedford-Stuyvesant,Brooklyn,Brooklyn,NY,11216,New York,US,United States,Apartment,Entire home/apt,6,1.0,2.0,3.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",100.0,2,$15.00,3,400,151,2014-08-09,2018-10-21,93.0,10.0,9.0,10.0,10.0,9.0,9.0,1.0,strict_14_with_grace_period,0.0,0.0,1,2.92,0,0
5,2015-07-11,US,1.0,100.0,0.0,Kips Bay,1.0,"['email', 'phone', 'reviews']",1.0,0.0,Kips Bay,Manhattan,New York,NY,10016,New York,US,United States,Apartment,Entire home/apt,2,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",200.0,2,$50.00,2,1125,75,2015-09-21,2018-10-26,95.0,10.0,9.0,10.0,10.0,10.0,9.0,0.0,strict_14_with_grace_period,0.0,0.0,1,1.97,0,0
6,2017-05-22,"New York, New York, United States",1.0,100.0,0.0,Astoria,1.0,"['phone', 'reviews']",1.0,0.0,Astoria,Queens,Queens,NY,11103.0,New York,US,United States,Apartment,Private room,2,1.0,1.0,1.0,Real Bed,"{Wifi,""Air conditioning"",Kitchen,""Free street ...",60.0,1,$10.00,1,7,95,2017-06-11,2018-10-17,94.0,10.0,9.0,10.0,10.0,10.0,10.0,1.0,flexible,0.0,0.0,1,5.57,0,0
8,2015-07-20,"New York, New York, United States",0.0,86.0,0.0,Midtown East,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",1.0,1.0,Midtown,Manhattan,New York,NY,10022.0,New York,US,United States,Apartment,Entire home/apt,6,1.0,2.0,2.0,Real Bed,"{TV,""Cable TV"",Wifi,""Air conditioning"",Kitchen...",350.0,1,$0.00,3,14,1,2017-11-25,2017-11-25,80.0,10.0,8.0,10.0,10.0,8.0,8.0,1.0,moderate,0.0,0.0,1,0.09,1,1
9,2015-11-12,"Concord, California, United States",0.0,86.0,0.0,Hamilton Heights,1.0,"['email', 'phone', 'facebook', 'reviews', 'kba']",1.0,1.0,Harlem,Manhattan,New York,NY,10031.0,New York,US,United States,Apartment,Private room,2,1.0,1.0,1.0,Real Bed,"{Wifi,""Air conditioning"",""Pets allowed"",Elevat...",60.0,1,$0.00,3,14,1,2017-08-06,2017-08-06,80.0,8.0,8.0,10.0,10.0,10.0,8.0,0.0,flexible,0.0,0.0,1,0.07,1,1


In [122]:
# Encoded all binary features
# f = 0.0, t = 1.0

binary_features = ["host_is_superhost", "host_has_profile_pic", "host_identity_verified", "instant_bookable", "require_guest_profile_picture","require_guest_phone_verification"]
encoding = {"f": 0.0, "t": 1.0}

for feature in binary_features:
    trainDF[feature] = trainDF[feature].replace({"f": 0.0, "t": 1.0})


In [133]:
trainDF["bed_type"].value_counts()

bed_type
Real Bed         17299
Futon              144
Pull-out Sofa      116
Airbed              59
Couch               26
Name: count, dtype: int64

In [130]:
trainDF["neighbourhood_group_cleansed"].value_counts()

neighbourhood_group_cleansed
Manhattan        7793
Brooklyn         7513
Queens           1899
Bronx             318
Staten Island     121
Name: count, dtype: int64

In [131]:
trainDF["neighbourhood_cleansed"].value_counts()

neighbourhood_cleansed
Williamsburg                  1563
Bedford-Stuyvesant            1353
Harlem                        1055
Bushwick                       944
Hell's Kitchen                 747
Upper West Side                728
East Village                   727
Upper East Side                689
Crown Heights                  577
Midtown                        470
Chelsea                        427
East Harlem                    425
Greenpoint                     421
Lower East Side                339
Astoria                        331
Washington Heights             286
West Village                   279
Clinton Hill                   217
Flatbush                       216
Prospect-Lefferts Gardens      204
Park Slope                     196
Long Island City               196
Fort Greene                    167
Financial District             166
Sunset Park                    151
East Flatbush                  149
Prospect Heights               146
Greenwich Village              1

Data Quality Issues

In terms of data quality issues, this dataset has many missing/ NA values as well as context dependent data. For example, features such as "summary" and "description" are textual data, specific to each listing and can't be used as a comparison metric.

# 1. Model Training (60 points)

Train at least 3 models. You can evaluate more than 3 models and apply your own cross-validation approach to choose the top three models to submit to Kaggle. For each model, include a short write up on each model (desc of selected model, evaluation approach applied, how did you select hyperparameters).  

In [115]:
# Model 1 - Linear Regression

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression



[INSERT WRITE-UP HERE]

In [116]:
# Model 2

[INSERT WRITE-UP HERE]

In [117]:
# Model 3

[INSERT WRITE-UP HERE]

# 2. Plotting (10 points)

In [118]:
# Plot results here
# [INSERT CODE HERE]

[INSERT WRITE-UP HERE]

# 3. Discussion (10 points)

# include a short discussion about results and approach.

Note, the rest of the points are based on Kaggle submission and ranking.

# References / Disclosure

[List the sources you used to complete this assignment here]